# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from torch import optim
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [10]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

0it [00:00, ?it/s]

Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw


0it [00:00, ?it/s]

Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw


0it [00:00, ?it/s]

Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw


0it [00:00, ?it/s]

Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw
Processing...
Done!


/Users/fengjinghao/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1607370126481/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Then, we define the model, object function and optimizer that we use to classify.

In [26]:
class SimpleNet(nn.Module):
# TODO:define model

    def __init__(self):
        super(SimpleNet,self).__init__()
        
        self.conv1= nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        #print(x.shape)
        x = x.view(-1,self.num_flat_features(x))
        #print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    
model = SimpleNet()
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),1e-1)

Next, we can start to train and evaluate!

In [30]:
# train and evaluate
losses = []
acces = []
eval_losses = []
eval_acces = []

for epoch in range(NUM_EPOCHS):
    train_loss = 0
    train_acc = 0
    model = model.train()
    for images, labels in train_loader:
        # TODO:forward + backward + optimize
        images = Variable(images)
        labels = Variable(labels)
        
        #forwarding
        out = model(images)
        loss = criterion(out, labels)
        
        #backwarding
        optimizer.zero_grad()
        loss.backward(retain_graph = True)
        optimizer.step()
        
        #recording loss
        _,pred = out.max(1)
        num_correct = (pred == labels).sum().item()
        acc = num_correct / images.shape[0]
        train_acc += acc
        
    losses.append(train_loss / len(train_loader))
    acces.append(train_acc / len(train_loader))
    
    eval_loss = 0
    eval_acc = 0  
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    for img , label in test_loader:
        #img = img.reshape(img.size(0),-1)
        img = Variable(img)
        label = Variable(label)
        
        out = model(img)
        loss = criterion(out,label)
        # 记录误差
        eval_loss += loss.item()
        
        _ , pred = out.max(1)
        num_correct = (pred==label).sum().item()
        acc = num_correct / img.shape[0]
        
        eval_acc += acc
    eval_losses.append(eval_loss / len(test_loader))
    eval_acces.append(eval_acc / len(test_loader))
    
    print('Epoch {} Train  Accuracy {} Test Accuracy {}'.format(
        epoch+1, train_acc / len(train_loader), eval_acc / len(test_loader)))

Epoch 1 Train  Accuracy 0.9931390224358975 Test Accuracy 0.9893830128205128
Epoch 2 Train  Accuracy 0.9944911858974359 Test Accuracy 0.9873798076923077
Epoch 3 Train  Accuracy 0.9949586004273504 Test Accuracy 0.9897836538461539
Epoch 4 Train  Accuracy 0.9957431891025641 Test Accuracy 0.9896834935897436
Epoch 5 Train  Accuracy 0.9961104433760684 Test Accuracy 0.9884815705128205
Epoch 6 Train  Accuracy 0.9959101228632479 Test Accuracy 0.9892828525641025
Epoch 7 Train  Accuracy 0.9967280982905983 Test Accuracy 0.9889823717948718
Epoch 8 Train  Accuracy 0.9973624465811965 Test Accuracy 0.9901842948717948
Epoch 9 Train  Accuracy 0.997779780982906 Test Accuracy 0.9903846153846154
Epoch 10 Train  Accuracy 0.9976963141025641 Test Accuracy 0.9881810897435898


#### Q5:
Please print the training and testing accuracy.